<a href="https://colab.research.google.com/github/GuerrSim96/Molecular_Dynamics_Simulation_with_Trifluoroethanol/blob/main/trajectory_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title ##### Install and Import
!pip install MDAnalysis --quiet
import MDAnalysis as mda
from MDAnalysis.analysis import distances, diffusionmap, align
import os
import random
import shutil as sh
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.colors import LinearSegmentedColormap
from scipy.signal import savgol_filter #Smoothing using Savitzky-Golay
print('all the packages have been imported! ;)')

In [ ]:
#@title ##### Mount the Drive
#@markdown # Click the last output URL
!sudo add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!sudo apt-get update -qq 2>&1 > /dev/null
!sudo apt -y install -qq google-drive-ocamlfuse 2>&1 > /dev/null
!google-drive-ocamlfuse

In [ ]:
!sudo apt-get install -qq w3m # to act as web browser
!xdg-settings set default-web-browser w3m.desktop # to set default browser
%cd /content
!mkdir drive
%cd drive
!mkdir MyDrive
%cd ..
!google-drive-ocamlfuse /content/drive/MyDrive

In [ ]:
# Give everyone all permission
path_bin='/content/drive/MyDrive/GROMACS_SG/gromacs-2023/bin/'
for root, directories, files in os.walk(path_bin):
    for dir in directories:
        os.chmod(os.path.join(root, dir), 0o111)
    for f in files:
        os.chmod(os.path.join(root, f), 0o111)

In [ ]:
p='/content/drive/MyDrive/GROMACS_SG/md_files/'
fp= os.listdir(p)
print('List of former Molecular Dynamics:')
for x in fp:
  print(x)
pdb_dir= input('select the structure from the list above :')
p0=f'/content/drive/MyDrive/GROMACS_SG/md_files/{pdb_dir}/'
fp0=os.listdir(p0)
for y in fp0:
  print(y)
ff_dir= input('select the simulation from the list above :')
p1=p0+f'{ff_dir}/'
fp1=os.listdir(p1)
for file in fp1:
  if file.endswith('.tpr'):
    if file.startswith('md'):
      sh.copy(p1+file,'/content/')
    print('-The file \"'+ file +'\" has been moved successfully')
  elif file.endswith('protPBC.xtc'):
    if file.startswith('md'):
      sh.copy(p1+file,'/content/')
    print('-The file \"'+ file +'\" has been moved successfully')
  elif file.endswith('.edr'):
    if file.startswith('md'):
      sh.copy(p1+file,'/content/')
    print('-The file \"'+ file +'\" has been moved successfully')
  elif file.endswith('.gro'):
    if file.startswith('md'):
      sh.copy(p1+file,'/content/')
    print('-The file \"'+ file +'\" has been moved successfully')

In [ ]:
fc_1=os.listdir('/content')
open('options','x') # Creating the options file
for c in fc_1:
  if c.endswith('.xtc'):
    os.rename(c,'file.xtc')
    print(c+' has been renamed')
  elif c.endswith('.tpr'):
    os.rename(c,'file.tpr')
    print(c+' has been renamed')
  elif c.endswith('.edr'):
    os.rename(c,'md.edr')
    print(c+' has been renamed')
  elif c.endswith('.gro'):
    os.rename(c,'file.gro')
    print(c+' has been renamed')

In [ ]:
def gmx(cmd):
  '''To launch gmx process'''
  process = subprocess.Popen(cmd, shell=True, executable='/bin/bash', stdout=subprocess.PIPE, stderr=subprocess.PIPE)
  output, error = process.communicate()
  print(f'Output: \n{output.decode("utf-8")} \n\nError: \n{error.decode("utf-8")}\n\n')

In [ ]:
#@title # Energy
with open('options','w') as file: # Overwrite "options" file
  file.write('Potential \n')
# Energy of the structure during molecular dynamics
gmx('gmx energy -f md.edr -o potential_md.xvg -xvg none < options')
# Plot
data_md = np.loadtxt('potential_md.xvg')
yhat = savgol_filter(data_md[:,1], 21, 1)
plt.title('Energy of MD Simulation')
plt.xlabel('Time [ps]')
plt.ylabel('Potential Energy [kJ/mol]')
plt.plot(data_md[:,0], data_md[:,1], linestyle='solid', linewidth='0.5', color='black')
plt.plot(data_md[:,0], yhat, linestyle='solid', linewidth='0.7', color='red')
plt.savefig('potential_md.png', bbox_inches='tight')
plt.show()

In [ ]:
#@title # Compattezza e Flessibilità
with open('options','w') as file: # Overwrite "options" file
  file.write('Protein \n')
# Raggio d'inerzia
gmx('gmx gyrate -s file.tpr -f file.xtc -o gyrate.xvg -xvg none < options')
# SASA: compattezza
gmx('gmx sasa -s file.tpr -f file.xtc -o sasa.xvg -tu ns -xvg none < options')
# RMSF: per la flessibilità
gmx('gmx rmsf -s file.tpr -f file.xtc -o rmsf.xvg -xvg none -res < options')
# Plot
# Raggio d'inerzia durante la dinamica molecolare
data_gr = np.loadtxt('gyrate.xvg')
yhat = savgol_filter(data_gr[:,1], 21, 1)
plt.title('Radius of gyration')
plt.xlabel('Time [ps]')
plt.ylabel('Radius of gyration [nm]')
plt.plot(data_gr[:,0], data_gr[:,1], linestyle='solid', linewidth='0.5', color='black') # gyrate nella prima colonna
plt.plot(data_gr[:,0], yhat, linestyle='solid', linewidth='0.7', color='red')
plt.savefig('gyrate.eps', format='eps', bbox_inches='tight')
plt.show()
# SASA
sasa=np.loadtxt('sasa.xvg')
yhat = savgol_filter(sasa[:,1], 21, 1)
plt.title('Solvent Accessible Surface')
plt.xlabel('Time [ns]')
plt.ylabel('Area [nm$^{2}$]')
plt.plot(sasa[:,0], sasa[:,1], linestyle='solid', linewidth='0.5', color='black')
plt.plot(sasa[:,0], yhat, linestyle='solid', linewidth='0.7', color='red')
plt.tight_layout()
plt.savefig('sasa.eps', format='eps', bbox_inches='tight')
plt.show()
# RMSF
rmsf=np.loadtxt('rmsf.xvg')
plt.title('RMSF')
plt.xlabel('Res')
plt.ylabel('C-alpha RMSF [nm]')
plt.plot(rmsf[:,0], rmsf[:,1], linestyle='solid', linewidth='0.7', color='black')
plt.tight_layout()
plt.savefig('rmsf.eps', format='eps', bbox_inches='tight')
plt.show()

In [ ]:
#@title # RMSD
with open('options','w') as file:
  file.write('Backbone \n Backbone \n')
# RMSD: struttura minimizzata rispetto alla dinamica
gmx('gmx rms -s file.tpr -f file.xtc -o rmsd.xvg -tu ns -xvg none < options')
# Plot
#rmsd
rmsd_md=np.loadtxt('rmsd.xvg') #tra struttura minimizzata e dinamica
yhat = savgol_filter(rmsd_md[:,1], 21, 1)
plt.title('RMSD backbone')
plt.xlabel('Time [ns]')
plt.ylabel('RMSD [nm]')
plt.plot(rmsd_md[:,0], rmsd_md[:,1], linestyle='solid', linewidth='0.7', color='black')
plt.plot(rmsd_md[:,0], yhat, linestyle='solid', linewidth='0.9', color='red')
plt.tight_layout()
plt.savefig('rmsd.eps', format='eps', bbox_inches='tight')
plt.show()
#pairwise rmsd
u = mda.Universe('/content/file.gro','/content/file.xtc')
aligner= align.AlignTraj(u,u, select='name CA', in_memory=True).run()
matrix= diffusionmap.DistanceMatrix(u, select='name CA').run()
matrix.dist_matrix.shape
plt.imshow(matrix.dist_matrix, cmap='viridis')
plt.title('Pairwise RMSD')
plt.xlabel('Frame', fontsize=12)
plt.ylabel('Frame', fontsize=12)
plt.colorbar(label='RMSD')
plt.tight_layout()
plt.savefig('pairwise_rmsd.eps', format='eps', bbox_inches='tight')
plt.show()

In [ ]:
#@title # Matrice delle Distanze
#@markdown La cella in questione non è ancora stata ottimizzata ne commentata a dovere
frame_to_analyze= 10 #@param {type:"slider", min:10, max:5000, step:10}
os.environ['FRAME']= frame_to_analyze
with open('options','w') as file: # Overwrite "options" file
  file.write('Protein \n')
gmx('gmx trjconv -s file.tpr -f file.xtc -o output.pdb -dump "$FRAME" < options') #flag -dump to select the frame

u = mda.Universe('output.pdb')
ca = u.select_atoms('name CA')
n_ca = len(ca)
if len(ca)==0:
  print('There\'s a problem')
self_distances = distances.self_distance_array(ca.positions)

sq_dist_arr = np.zeros((n_ca, n_ca))
triu = np.triu_indices_from(sq_dist_arr, k=1)
sq_dist_arr[triu] = self_distances
sq_dist_arr.T[triu] = self_distances

fig, ax = plt.subplots()
im = ax.pcolor(ca.resids, ca.resids, sq_dist_arr)

# plt.pcolor gives a rectangular grid by default
# so we need to make our heatmap square
ax.set_aspect('equal')

# add figure labels and titles
plt.ylabel('Residue IDs')
plt.xlabel('Residue IDs')
plt.title('Distance between alpha-carbons')

# colorbar
cbar = fig.colorbar(im)
cbar.ax.set_ylabel('Distance (Angstrom)')

In [ ]:
#@title # DSSP
gmx('gmx dssp -f file.xtc -s file.tpr -o dssp.dat -xvg none -hmode gromacs')
# dssp
row, leg_0, num_row= [], [], []
num_dict={'~':'0', 'G':'1', 'H':'2', 'I':'3', 'T':'4', 'E':'5', 'B':'6', 'S':'7', 'C':'8', 'P':'9'} # Dictionary
col_dict={'0':'gray', '1':'red', '2':'orange', '3':'yellow', '4':'pink', '5':'blue', '6':'cyan', '7':'purple', '8':'brown', '9':'brown'}
with open('dssp.dat', 'r') as file: # open file output for obtein the information
  for line in file:
    x=list(line)
    row.append(x) # list of the letter present in line
for ss in row:
  ss.remove('\n') # remove the special character "\n"
for lst in row:
  for e in lst:
    leg_0.append(e) # Transformation of "row", which is a list of multiple list, in "leg_0" a simple list
leg_1=set(leg_0)    # Using "set()" function you'll get all the characters thet are in a list
leg_2=list(leg_1)   # Transforming the set in a list
lst_n = list(range(len(leg_2))) # Get a list of colors long as the another list
for sublist in row:
  sublist_conv=[]
  for l in sublist:                     # for the elements of the lists inside the major list "row"
    sublist_conv.append(int(num_dict[l]))   # It allows to convert the elements in numbers thanks the association of the dictionary
  num_row.append(sublist_conv)         # Recreate the major list converted which means that there are not characters but only numbers
data=pd.DataFrame(num_row)
colors= [col_dict[f'{value}'] for value in lst_n]
cmap_c= LinearSegmentedColormap.from_list('custom_cmap', colors)
# Text to use as a legend
with open('legend_dssp','w') as file: # File for the legend of the plot
  file.write('LEGEND: \nG = 310-helix \nH = Alpha-helix \nI = Pi-helix \nT = Turn \nE = Beta-sheet \nB = Beta-bridge \nS = Bend \nC or P = Coil \n~ = Unknown or missing\n')
with open('legend_dssp', 'a') as file:
  file.write('\nCHAR TO COL: \n')
  for value in num_dict:
    file.write(f'{value} = {col_dict[num_dict[value]]}\n')
with open('legend_dssp', 'r') as file:
  legend = file.read() # Information about how to read the plot
# plot of secondary structures
fig = plt.figure(figsize=(50, 10))
ax = fig.add_subplot(111)
ax.set_title('Secondary Structure')
data_array = np.transpose(data)
im = ax.imshow(data_array, cmap= cmap_c, aspect='auto')
ax.set_xticks(np.arange(data.shape[0]))
ax.set_xticklabels(data.index, fontsize=0.1)
ax.set_yticks(np.arange(data.shape[1]))
ax.set_yticklabels(data.columns)
plt.xlabel('Time')
plt.ylabel('Residue')
plt.text(100000, 18, legend, fontsize=12, color='black', va='center', ha='left')
plt.tight_layout()
# plt.savefig('dssp_res.eps', format='eps', bbox_inches='tight')
plt.show()

In [ ]:
#@title # RAMA
gmx('gmx rama -f file.xtc -s file.tpr -o rama.xvg -xvg none')
#rama
rama= pd.read_csv('rama.xvg', delimiter='  ', names=['phi', 'psi', 'res']) # Trasforming the file in a DataFrame imposing 3 columns to it
rama[['res', 'num']] = rama['res'].str.split('-', expand=True) # Slitting the "res" column to get also the "num" column
rama['num'] = rama['num'].astype(int) # Forcing the values to be integer
groups= rama.groupby('num') # Grouping by "num" values
rama_4num = []
for number_group, data in groups: # Generate a list of DataFrame one for group
  df = pd.DataFrame(data)
  rama_4num.append(df)
num_plot, min, max = len(rama_4num), -180, 180 # Defining some variables
fig, axs = plt.subplots(int(num_plot/4)+1, 4, figsize=(int(num_plot/2), num_plot))
axs = axs.flatten() # Forcing the array on one axis
for i, df in enumerate(rama_4num): # Scatter plot
  ax = axs[i]
  ax.scatter(df['phi'],df['psi'], s=10, alpha=0.25, color='black')
  ax.set_title(f'Torsion Angles {df["res"].iloc[0]}-{df["num"].iloc[0]}')
  ax.set_xlabel(df.columns[0])
  ax.set_ylabel(df.columns[1])
  ax.set_xlim(min, max)
  ax.set_ylim(min, max)
for j in range(num_plot, len(axs)): # Deleting the empty subplot
  fig.delaxes(axs[j])
plt.tight_layout()
plt.savefig('rama_num.eps', format='eps', bbox_inches='tight') # Saving the plot as a vectorial image
plt.show()
groups_r= rama.groupby('res') # Grouping by "res" values
rama_4res = []
for number_group, data in groups_r: # Generate a list of DataFrame one for group
  df = pd.DataFrame(data)
  rama_4res.append(df)
num_plot_r= len(rama_4res)
fig, axs = plt.subplots(5, int(num_plot_r/4)+1, figsize=( num_plot_r, num_plot_r))
axs = axs.flatten() # Forcing the array on one axis
for i, df in enumerate(rama_4res):
  ax = axs[i]
  ax.scatter(df['phi'],df['psi'], s=10, alpha=0.25, color='black')
  ax.set_title(f'Torsion Angles {df["res"].iloc[0]}')
  ax.set_xlabel(df.columns[0])
  ax.set_ylabel(df.columns[1])
  ax.set_xlim(min, max)
  ax.set_ylim(min, max)
for j in range(num_plot_r, len(axs)): # Deleting the empty subplot
  fig.delaxes(axs[j])
plt.tight_layout()
plt.savefig('rama_res.eps', format='eps', bbox_inches='tight') # Saving the plot as a vectorial image
plt.show()

In [ ]:
#@title # H-bond
#@markdown Produzione del plot ancora da ultimare
with open('options','w') as file:
  file.write('Protein \n Protein \n')
gmx('gmx hbond -f file.xtc -s file.tpr -hx hbhelix.xvg -xvg none -tu ns < options')

#Export

In [ ]:
def pa_di(list_path):
  '''Funzione per creare nuove cartelle all'interno del drive a partire da una lista di directory.'''
  print(pa_di.__doc__)
  for dir in list_path:
    if os.path.isdir(dir) is True:
      print('-The directory \"'+ dir +'\" is already there')
    else:
      os.mkdir(dir)
      print('-The directory \"'+ dir +'\" has been created successfully')

In [ ]:
lst_dir= []
#Sottodirectory per separare gli output
path_eps, path_xvg= p1+'eps/', p1+'xvg/'
lst_dir.append(path_eps)
lst_dir.append(path_xvg)
pa_di(lst_dir)

In [ ]:
fc_2, files_xvg, files_eps=os.listdir('/content'), os.listdir(path_xvg), os.listdir(path_eps)
for file in fc_2:
  if file.endswith('.xvg'):
    if file not in files_xvg:
      sh.copy('/content/'+ file, path_xvg)
      print('-The file \"'+ file +'\" has been moved successfully')
    else:
      print('-The file \"'+ file +'\" is already there')
  elif file.endswith('.eps'):
    if file not in files_eps:
      sh.copy('/content/'+ file, path_eps)
      print('-The file \"'+ file +'\" has been moved successfully')
    else:
      print('-The file \"'+ file +'\" is already there')